# Obtaining SimpleMaxwell Thorn
Follow the instructions at [here](https://bitbucket.org/yosef_zlochower/tutorial/src/master/)

# Defining grid functions

Suppose we want to define four gridfunction, f, dx_f, dy_f, and dz_f, where the latter three are going to represent the gradient of the first. In our interface.ccl, we would add
```
CCTK_REAL my_scalar TYPE=GF TIMELEVELS=3 TAGS='tensortypealias="Scalar"'
{
  f
} "scalar fields"

CCTK_REAL my_gradient TYPE=GF TIMELEVELS=3 TAGS='tensortypealias="U"'
{
  dx_f, dy_f, dz_f
} "Gradient field"
```
"my_scalar" and "my_gradient" are the group names for these gridfunctions (groups are collections of gridfunction). We need to tell cactus that the first is a scalar and the second a vector so that symmetry boundary conditions can be automatically applied

We indicate when storage is allocated for the gridfunctions in the schedule.ccl file, for example
```
STORAGE: my_scalar[3]
STORAGE: my_gradient[3]
```



## Using gridfunctions: Derivatives
Cactus stores 3-dimensional arrays as 1-dimensional arrays in Fortran ordering. The logical dimension of gridfunctions are stored in the array, cctk_lsh.

|Array Index | Logical Index (i, j, k)
| --- | --- |
| 0 | (0,0,0) |
| 1 | (1,0,0) |
| 2 | (2,0,0) |
| ... | ...|
| nx-1 | (nx -1, 0,0)|
| padding|
| nx + padding| (0,1,0) |
| ... | ... |
| | (nx-1, ny-1, 0)|
| padding |
| | (0,0, 1)|

To relate the logical indices (i,j,k) with the array index, use the Macro <code> CCTK_GFINDEX3D(cctkGH, i, j, k)</code>, i.e.,
<code>val = array[CCTK_GFINDEX3D(cctkGH, i, j, k)]</code>

We can therefore use the following code to calculate derivative, assuming that f, dx_f, dy_f, and dz_f are gridfunctions

```C
/* grid spacing in x */
const CCTK_REAL dx = CCTK_DELTA_SPACE(0);
/* grid spacing in x */
const CCTK_REAL dy = CCTK_DELTA_SPACE(1);
/* grid spacing in x */
const CCTK_REAL dz = CCTK_DELTA_SPACE(2);
/* Stride in x */
const CCTK_INT di = CCTK_GFINDEX3D(cctkGH, 1, 0, 0)
                   - CCTK_GFINDEX3D(cctkGH, 0, 0, 0);
/* Stride in y */
const CCTK_INT dj = CCTK_GFINDEX3D(cctkGH, 0, 1, 0) 
                   - CCTK_GFINDEX3D(cctkGH, 0, 0, 0);
/* Stride in z */
const CCTK_INT dk = CCTK_GFINDEX3D(cctkGH, 0, 0, 1)
                   - CCTK_GFINDEX3D(cctkGH, 0, 0, 0);
   
for (int k = 1; k < cctk_lsh[2] - 1; k++)
{
  for (int j = 1; j < cctk_lsh[1] - 1; j++)
  {
    for (int i = 1; i < cctk_lsh[0] - 1; i++)
    {
      const CCTK_INT indx = CCTK_GFINDEX3D(cctkGH, i, j, k);
      dx_f[indx] = 0.5 * (f[indx + di] - f[indx - di])/dx;
      dy_f[indx] = 0.5 * (f[indx + dj] - f[indx - dj])/dy;
      dz_f[indx] = 0.5 * (f[indx + dk] - f[indx - dk])/dz;
    }
  }
}
```
Note that the loop bounds skip the first and last elements in all directions.

# Design choices: Use of macros

Both the interior and boundary algorithms will need to be implemented for various finite difference operators, e.g., centered fourth-order, centered second-order, and first-order derivatives are all needed. Rather than recoding the same basic algorithm many times, the algorithm will be implemented as a MACRO that, in turns, used additional MACROS to implement the derivatives.

For example, the following code implements the derivative operation above and a second one

```
#define MAIN_ALGORITHM \
  dx_f[indx] = DIFFX(f);\
  dy_f[indx] = DIFFY(f);\
  dz_f[indx] = DIFFZ(f);

#define D2CEN(f_, i_, di_, h_) \
  ((-f_[-(di_) + (i_)] + f_[(di_) + (i_)]) * oo2##h_)

#define D4CEN(f_, i_, di_, h_)                                                 \
  ((f_[-2 * (di_) + (i_)] - 8 * f_[-(di_) + (i_)] + 8 * f_[(di_) + (i_)] -     \
    f_[2 * (di_) + (i_)]) *                                                    \
   oo12##h_)   
#define DIFFX(f_) D2CEN(f_, ijk, di, dx)
#define DIFFY(f_) D2CEN(f_, ijk, dj, dy)
#define DIFFZ(f_) D2CEN(f_, ijk, dk, dz)

const double oo2dx = 0.5 / dx;
const double oo2dy = 0.5 / dy;
const double oo2dz = 0.5 / dz;

for (int k = 1; k < cctk_lsh[2] - 1; k++)
{
  for (int j = 1; j < cctk_lsh[1] - 1; j++)
  {
    for (int i = 1; i < cctk_lsh[0] - 1; i++)
    {
      const CCTK_INT ijk = CCTK_GFINDEX3D(cctkGH, i, j, k);
      MAIN_ALGORITHM
    }
  }
}
#undef DIFFX
#undef DIFFY
#undef DIFFZ
#define DIFFX(f_) D4CEN(f_, ijk, di, dx)
#define DIFFY(f_) D4CEN(f_, ijk, dj, dy)
#define DIFFZ(f_) D4CEN(f_, ijk, dk, dz)

const double oo12dx = 1.0 / (12 * dx);
const double oo12dy = 1.0 / (12 * dy);
const double oo12dz = 1.0 / (12 * dz);

for (int k = 2; k < cctk_lsh[2] - 2; k++)
{
  for (int j = 2; j < cctk_lsh[1] - 2; j++)
  {
    for (int i = 2; i < cctk_lsh[0] - 2; i++)
    {
      const CCTK_INT ijk = CCTK_GFINDEX3D(cctkGH, i, j, k);
      MAIN_ALGORITHM
    }
  }
}
```

The above gets expanded to
```
const double oo2dx = 0.5 / dx;
const double oo2dy = 0.5 / dy;
const double oo2dz = 0.5 / dz;

for (int k = 1; k < cctk_lsh[2] - 1; k++)
{
  for (int j = 1; j < cctk_lsh[1] - 1; j++)
  {
    for (int i = 1; i < cctk_lsh[0] - 1; i++)
    {
      const CCTK_INT ijk = CCTK_GFINDEX3D(cctkGH, i, j, k);
      dx_f[indx] = ((-f[-(di) + (ijk)] + f[(di) + (ijk)]) * oo2dx);
      dy_f[indx] = ((-f[-(dj) + (ijk)] + f[(dj) + (ijk)]) * oo2dy);
      dz_f[indx] = ((-f[-(dk) + (ijk)] + f[(dk) + (ijk)]) * oo2dz);
    }
  }
}

const double oo12dx = 1.0 / (12 * dx);
const double oo12dy = 1.0 / (12 * dy);
const double oo12dz = 1.0 / (12 * dz);

for (
int k = 2; k < cctk_lsh[2] - 2; k++)
{
  for (int j = 2; j < cctk_lsh[1] - 2; j++)
  {
    for (int i = 2; i < cctk_lsh[0] - 2; i++)
    {
      const CCTK_INT ijk = CCTK_GFINDEX3D(cctkGH, i, j, k);
      dx_f[indx] = ((f[-2 * (di) + (ijk)] - 8 * f[-(di) + (ijk)] +
                     8 * f[(di) + (ijk)] - f[2 * (di) + (ijk)]) *
                    oo12dx);
      dy_f[indx] = ((f[-2 * (dj) + (ijk)] - 8 * f[-(dj) + (ijk)] +
                     8 * f[(dj) + (ijk)] - f[2 * (dj) + (ijk)]) *
                    oo12dy);
      dz_f[indx] = ((f[-2 * (dk) + (ijk)] - 8 * f[-(dk) + (ijk)] +
                     8 * f[(dk) + (ijk)] - f[2 * (dk) + (ijk)]) *
                    oo12dz);
    }
  }
}

```

In addition, we will use sympy to help generate the macros that implement both the interior and boundary algorithms


# Design choice: Outer boundary conditions

The Maxwell evolution system is of the form
\begin{equation}
  \partial_t {\bf U} = {\bf A}^x \partial_x {\bf U} + {\bf A}^y \partial_y {\bf U} + {\bf A}^z \partial_z {\bf U} +{\bf S}.
\end{equation}
Where ${\bf U}$ is a _vector_ containing all the evolved fields, ${\bf A}^x$, ${\bf A}^y$, and ${\bf A}^z$ are square symmetric matrices, and ${\bf S}$ is a vector. We will apply our boundary conditions to gridfunctions that contain $\partial_t {\bf U}$

# Appendix: Cactus variables and Macros Used in SimpleMaxwell

| symbol | type | use / meaning |
| --- | --- | --- |
| CCTK_ARGUMENTS | macro | argument list for functions called by cactus |
| cctk_lsh[3] | array of ints | local grid dimensions excluding padding |
| cctk_ash[3] | array of ints | local grid dimensions including padding |
| cctk_bbox[6] | array of ints | is local grid boundary the domain boundary |
| CCTK_DELTA_SPACE() | macro takes int as input| grid spacing per direction |
| CCTK_DELTA_TIME | macro | time step size |
| DECLARE_CCTK_ARGUMENTS | macro | declares GFs and several useful cactus variables |
| DECLARE_CCTK_PARAMETERS | macro | declares parameters |
| CCTK_REAL | type | default real type (typically double)|
| CCTK_INT | type | default int type (typically int)|
| CCTK_GFINDEX3D() | macro | converts logical i,j,k indices to flat array index |
| CCTK_VarIndex | function | determines GF ID based on its name |
| MoLRegisterEvolved | function | registers a GF to be evolved with MoL |
| MoLRegisterConstrained | function | registers a GF as a constrained variable with MoL |
| CCTK_WARN | wrapper to function | emit a warning message |
| CCTK_WARN_ABORT | macro | argument to CCTK_WARN, causes program termination|
| CCTK_WARN_ALERT | macro | argument to CCTK_WARN, indicates warning is not fatal|
| CCTK_INFO | wrapper to function | emit an informational message |
| Boundary_SelectGroupForBC | function | Register a group of GFs for boundary conditions the next time ApplyBCs is called


The boundaries of the grid on any given processor may or may not be part of the domain boundary. If you algorithm changes near the domain boundaries, then you can use cctk_bbox[] to determine which of local boundaries are part of the domain boundaries.

If `cctk_bbox[0] == 1` then the lower x boundary is part of the domain boundary. <br> 
If `cctk_bbox[1] == 1` then the upper x boundary is part of the domain boundary. <br> 
If `cctk_bbox[2] == 1` then the lower y boundary is part of the domain boundary. <br> 
If `cctk_bbox[3] == 1` then the upper y boundary is part of the domain boundary. <br> 
If `cctk_bbox[4] == 1` then the lower z boundary is part of the domain boundary. <br> 
If `cctk_bbox[5] == 1` then the upper z boundary is part of the domain boundary. <br>


# Appendix: Scheduling bins and groups used in SimpleMaxwell

Note that schedule groups are scheduled inside of schedule bins. Routines or groups are scheduled *at* bins, while routines or other groups are schedule *in* a group.

`CCTK_BASEGRID`: Scheduling bin for setting up coordinates and perforing initializations that need to be done each time a simulation restart.<br> 
`CCTK_INITIAL`: Scheduling bin for initial data. Routines scheduled here only run at the start of a simulation (iteration 0).<br>

`CCTK_POSTRESTRICTINITIAL`: Scheduling bin for routines that need to be run after a _restriction_ operation at the initial time step (iteration 0). Only relevant if mesh refinement is used.
<br>
`CCTK_POSTRESTRICT`: Scheduling bin for routines that need to be run after a _restriction_ operation after iteration 0. Only relevant if mesh refinement is used.
<br>
`CCTK_POSTREGRIDINITIAL`: Scheduling bin for routines that need to be run after a _regridding_ operation during iteration 0. Only relevant if mesh refinement is used.
<br>
`CCTK_POSTREGRIDINITIAL`: Scheduling bin for routines that need to be run after a _regridding_ operation after iteration 0. Only relevant if mesh refinement is used.
<br>
<br>
`MoL_CalcRHS`: A schedule group for routines that fill in RHS variables.
<br>
`MoL_PostStep`: A schedule group for routines that need to run after a MoL ministep completes. Examples would be routines implement algebraic constraints in BSSN.
<br>
`MoL_PseudoEvolution`: A schedule group for routines that update constrained variables.

Consider
```
SCHEDULE GROUP SimpleMaxwell_PostMiniStep IN MoL_PostStep
{                                           
} "Schedule group for symmetries and others"


SCHEDULE SimpleMaxwell_Boundaries IN SimpleMaxwell_PostMiniStep
{               
  OPTIONS: LEVEL                                      
  SYNC: EvolvedVectorD, EvolvedVectorB, EvolvedScalars
  LANG: C                                
} "Schedule symmetry boundary conditions"

```
The function SimpleMaxwell_Boundaries is being scheduled within the group SimpleMaxwell_PostMiniStep, which, in turn, is scheduled within the group MoL_PostStep, which, in turn (although not apparent from the above) is ultimately scheduled *at* several diffrent CCTK bins.